In [1]:
!pip install pytorchvideo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 KB 579.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 KB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 20.9 MB/s eta 0:00:00
  Created wheel for pytorchvideo: filename=pytorchvideo-0.1.5-py3-none-any.whl size=188714 sha256=7af5d2ae7fa70efbce0f3a464802664bb6084bf0aea4f13f67c4c9aa0b538b4a
  Stored in directory: /root/.cache/pip/wheels/e8/51/05/053b29bac2400cbbae2fb7cfc41afd280d627bca7c9363ca80
  Created wheel for fvcore: filename=fvcore-0.1.5.post20220506-py3-none-any.whl size=61284 sha256=0054a76a5e3185270815b4e18528b0a0fe7fa9d5757f46bb3bdb705dc72ea379
  Stored in directory: /root/.cache/pip/wheels/f5/ef/3c/708de8799f89f0871bd209866831fe3885db93fa090608fa73
Successfully built pytorchvideo fvcore


In [2]:
import torch
import numpy as np
from torch.utils.data import (
    Dataset,
    DataLoader,
) 
import pickle

import os

import math
import torch.nn as nn
import torch.nn.functional as F

from torchvision.transforms import Compose, Lambda, Grayscale,Normalize
from torchvision.transforms._transforms_video import CenterCropVideo, NormalizeVideo
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    UniformTemporalSubsample,
)

from tqdm import tqdm

from collections import OrderedDict

import torch.optim as optim



class SignDataset(Dataset):
    def __init__(self,x,y):
        self.x = x
        self.y = y


    def __len__(self):

        return len(self.y)

        # length = 0
        # with open(self.file, 'rb') as f:
        #     data = pickle.load(f)

        # for x in data:
        #     length += len(data[x])
        # return length


    def __getitem__(self, index):
        return self.x[index], self.y[index]
        

/opt/conda/lib/python3.7/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The _functional_video module is deprecated. Please use the functional module instead.
  "The _functional_video module is deprecated. Please use the functional module instead."
/opt/conda/lib/python3.7/site-packages/torchvision/transforms/_transforms_video.py:26: UserWarning: The _transforms_video module is deprecated. Please use the transforms module instead.
  "The _transforms_video module is deprecated. Please use the transforms module instead."


In [3]:
IMAGE_HEIGHT = 720
IMAGE_WIDTH = 800
IMAGE_CHANNEL = 1
NUM_FRAMES = 25
NUM_CLASSES = 60



inputs =[] #x
classes = [] #y

def transform_data(x, mean, std):
    
    transform =  ApplyTransformToKey(
        key="video",
        transform=Compose(
            [

                Lambda(lambda x: x.permute(1,0,2,3)),#(frames(depth), channel, height, width) -> (channel, frames(depth), height, width)

                UniformTemporalSubsample(NUM_FRAMES),
                Lambda(lambda x: x.permute(1,0,2,3)),#(frames(depth), channel, height, width)
                Lambda(lambda x: x/255.0),
                
                Normalize((mean,), (std,)),

                CenterCropVideo([720,800]),
                Lambda(lambda x: x.permute(1,0,2,3)),#(channel, frames(depth), height, width)

            ]

        ),
    )
    
    return transform(x)



    

def get_data_info(f):
    for line in f:
        a = line.split(',')
        yield a
        


def load_dataloader(batch_size):
    video = EncodedVideo.from_path("../input/signdataset/p01_n000.mp4")
   


    with open('../input/signdataset/p01_n000.txt') as f: 
        for x in get_data_info(f):
            classes.append(int(x[0]))
            start_time = x[1]
            end_time = x[2]
            
            video_data = video.get_clip(start_sec=float(start_time)/1000.0, end_sec=float(end_time)/1000.0)

            
            video_data["video"] = Grayscale(num_output_channels=1)((video_data["video"]).permute(1,0,2,3))
#             video_data["video"] = video_data["video"]/255
            #print(video_data["video"].shape)
            
            std, mean = torch.std_mean(video_data["video"])
            std = std/255.0
            mean = mean/255.0
            print(std, mean)
            
            
            video_data = transform_data( video_data, mean, std)

        # Move the inputs to the desired device
            inputs.append(video_data["video"])

    signds = SignDataset(inputs, classes)
    dataloader = DataLoader(signds, batch_size=batch_size, shuffle=True, num_workers=1)

    return dataloader
        



In [4]:
class conv_3d(nn.Module):
    def __init__(self):
        super(conv_3d, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv3d(1, 64, kernel_size=5, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2)))
        
        self.conv2 = nn.Sequential(
            nn.Conv3d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2)))
        

        self._features = nn.Sequential(
            self.conv1,
            self.conv2
        )


    def forward(self, x):
        out = self._features(x)
        print("conv3d", out.shape)
        out= out.reshape(out.shape[0], out.shape[1]*out.shape[2], out.shape[3], out.shape[4])
        print("reshape conv3d ",out.shape)
        return out

class conv_2d(nn.Module):
    def __init__(self):
        super(conv_2d, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(640, 640, kernel_size = 3, padding =1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)))
            
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(640, 512, kernel_size = 3, padding =1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)))
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(512, 256, kernel_size = 3, padding =1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)))
        
    
    def forward(self,x):
        out = self.conv2(self.conv1(x))
        out = self.conv3(out)
       
        out = out.view(out.shape[0],-1)
       
        return out


  

In [5]:



n_classes = 60

model = nn.Sequential(OrderedDict([
    ('frontend', conv_3d()),
    ('mid', conv_2d()),
    ('fc', nn.Sequential( nn.Dropout(p=0.4), nn.Linear(135168, 1024),nn.Linear(1024,256), nn.Linear(256,60) ))
]))




# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()
# specify optimizer and learning rate
optimizer = optim.SGD(
  [
        {"params": model.fc.parameters(), "lr": 1e-3},
        {"params": model.mid.parameters(), "lr": 1e-5},
        {"params": model.frontend.parameters(), "lr": 1e-4},
  ],
  momentum = 0.9
)


def train(model, device, train_loader, optimizer, criterion, epoch):
    model.cuda()
    model.train()
    train_loss = 0
    correct = 0
    
    
    for epoch in tqdm(range(epoch)):
        num_samples = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            
            data = data.to(device)
            target = torch.tensor(target).to(device)

    
    
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            print(output.shape)
            print(output)
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            print(target)
            print(pred)
            num_samples += pred.shape[0]
            correct += int(pred==target)
            print("correct till now:",correct)
        
    train_loss /= num_samples
    print('Epoch: {} , Training Accuracy: {}/{} ({:.0f}%) Training Loss: {:.6f}'.format(
                epoch, correct, num_samples,
                100. * correct / num_samples, train_loss))


dataloader = load_dataloader(batch_size=1)




tensor(0.1398) tensor(0.4332)
tensor(0.1438) tensor(0.4375)
tensor(0.1520) tensor(0.4283)
tensor(0.1510) tensor(0.4327)
tensor(0.1509) tensor(0.4258)
tensor(0.1456) tensor(0.4366)
tensor(0.1496) tensor(0.4301)
tensor(0.1520) tensor(0.4315)
tensor(0.1514) tensor(0.4299)
tensor(0.1523) tensor(0.4298)
tensor(0.1563) tensor(0.4214)


In [6]:
x, y = next(iter(dataloader))

In [7]:
print(x)

tensor([[[[[0.4129, 0.4129, 0.4129,  ..., 0.4129, 0.4129, 0.4129],
           [0.4129, 0.4129, 0.4129,  ..., 0.4129, 0.4129, 0.4129],
           [0.4129, 0.4129, 0.4129,  ..., 0.4129, 0.4129, 0.4129],
           ...,
           [0.4129, 0.4129, 0.4129,  ..., 0.4129, 0.4129, 0.4129],
           [0.4129, 0.4129, 0.4129,  ..., 0.4129, 0.4129, 0.4129],
           [0.4129, 0.4129, 0.4129,  ..., 0.4129, 0.4129, 0.4129]],

          [[0.4129, 0.4129, 0.4129,  ..., 0.4129, 0.4129, 0.4129],
           [0.4129, 0.4129, 0.4129,  ..., 0.4129, 0.4129, 0.4129],
           [0.4129, 0.4129, 0.4129,  ..., 0.4129, 0.4129, 0.4129],
           ...,
           [0.4129, 0.4129, 0.4129,  ..., 0.4129, 0.4129, 0.4129],
           [0.4129, 0.4129, 0.4129,  ..., 0.4129, 0.4129, 0.4129],
           [0.4129, 0.4129, 0.4129,  ..., 0.4129, 0.4129, 0.4129]],

          [[0.4129, 0.4129, 0.4129,  ..., 0.4129, 0.4129, 0.4129],
           [0.4129, 0.4129, 0.4129,  ..., 0.4129, 0.4129, 0.4129],
           [0.4129, 0.4129

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
train(model,device,dataloader,optimizer,criterion,10)

cuda


  0%|          | 0/10 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


conv3d torch.Size([1, 128, 5, 179, 199])
reshape conv3d  torch.Size([1, 640, 179, 199])
torch.Size([1, 60])
tensor([[ 0.0037,  0.0348,  0.0522, -0.0027, -0.0169, -0.0200, -0.0066, -0.0390,
         -0.0001, -0.0310,  0.0245,  0.0342, -0.0290, -0.0347,  0.0166, -0.0217,
          0.0033, -0.0539,  0.0186,  0.0603, -0.0263,  0.0461, -0.0558, -0.0601,
         -0.0614,  0.0361, -0.0529, -0.0609,  0.0179, -0.0180, -0.0561, -0.0183,
          0.0207,  0.0493,  0.0065, -0.0064, -0.0214, -0.0121,  0.0348, -0.0365,
         -0.0357, -0.0502, -0.0277, -0.0449,  0.0098, -0.0023, -0.0569, -0.0389,
          0.0664,  0.0129, -0.0450,  0.0491,  0.0561, -0.0336, -0.0312, -0.0288,
         -0.0204, -0.0506,  0.0076,  0.0578]], device='cuda:0',
       grad_fn=<AddmmBackward>)
tensor([50], device='cuda:0')
tensor([[48]], device='cuda:0')
correct till now: 0
conv3d torch.Size([1, 128, 5, 179, 199])
reshape conv3d  torch.Size([1, 640, 179, 199])
torch.Size([1, 60])
tensor([[ 0.0071,  0.0346,  0.0538,  0.

 10%|█         | 1/10 [00:22<03:18, 22.06s/it]

torch.Size([1, 60])
tensor([[-0.0189,  0.0303,  0.0622, -0.0012, -0.0179, -0.0432, -0.0285, -0.0485,
         -0.0119, -0.0336,  0.0236,  0.0260, -0.0409, -0.0623,  0.0112, -0.0440,
          0.0010,  0.0321,  0.0129,  0.0423, -0.0294,  0.0351, -0.0612, -0.0493,
         -0.0673,  0.0371, -0.0685, -0.0191,  0.0044, -0.0177, -0.0699, -0.0227,
          0.0055,  0.1349, -0.0038, -0.0237, -0.0213, -0.0052,  0.0393, -0.0310,
         -0.0549, -0.0428, -0.0250, -0.0452, -0.0139, -0.0228, -0.0768, -0.0432,
          0.0593,  0.0130,  0.0116,  0.0458,  0.0402, -0.0393,  0.0164, -0.0376,
         -0.0372, -0.0601,  0.0162,  0.0284]], device='cuda:0',
       grad_fn=<AddmmBackward>)
tensor([54], device='cuda:0')
tensor([[33]], device='cuda:0')
correct till now: 0
conv3d torch.Size([1, 128, 5, 179, 199])
reshape conv3d  torch.Size([1, 640, 179, 199])
torch.Size([1, 60])
tensor([[-0.0315,  0.0238,  0.0647, -0.0136, -0.0163, -0.0558, -0.0444, -0.0596,
         -0.0050, -0.0276,  0.0275,  0.0317, -

...

torch.Size([1, 60])
tensor([[-1.1830, -0.8697, -0.3387, -0.9125, -0.2020, -1.5432, -1.0111, -0.3246,
         -0.6581, -0.4033, -0.4146, -0.4214, -0.8128, -1.4629, -0.5402, -0.9550,
         -0.5390,  4.1505, -0.6409, -0.8945, -0.0741, -0.4865, -0.9320, -0.3602,
         -0.6910, -0.3410, -0.8594,  3.8795, -1.1848, -0.3007, -0.8395, -0.7039,
         -0.6472,  3.8871, -0.6574, -0.8206, -0.3752,  2.8044, -0.4494,  0.0545,
         -1.2556,  4.0452, -0.1144, -0.5795, -1.0257, -0.7071, -1.0278, -0.8038,
         -0.6118, -0.5412,  3.9154, -0.2679, -0.7490, -0.4937,  4.6921, -0.7337,
         -0.6978, -0.9532, -0.3885, -1.0732]], device='cuda:0',
       grad_fn=<AddmmBackward>)
tensor([17], device='cuda:0')
tensor([[54]], device='cuda:0')
correct till now: 16
conv3d torch.Size([1, 128, 5, 179, 199])
reshape conv3d  torch.Size([1, 640, 179, 199])
torch.Size([1, 60])
tensor([[-1.0946, -0.8235, -0.3061, -0.8540, -0.1872, -1.4416, -0.9363, -0.3001,
         -0.6235, -0.3807, -0.3851, -0.3742, 

 90%|█████████ | 9/10 [02:31<00:16, 16.24s/it]

torch.Size([1, 60])
tensor([[-1.1821, -0.8492, -0.3131, -0.8696, -0.3276, -1.4572, -1.0160, -0.3984,
         -0.6898, -0.4418, -0.3936, -0.3829, -0.7690, -1.4074, -0.5759, -0.9000,
         -0.5793,  4.7209, -0.6790, -0.7709, -0.1362, -0.5660, -0.8889, -0.3763,
         -0.6337, -0.3657, -0.9143,  3.7199, -1.1853, -0.3076, -0.8850, -0.6263,
         -0.7002,  4.5841, -0.6320, -0.8672, -0.4060,  3.8641, -0.5044, -0.0582,
         -1.3112,  3.4713, -0.1565, -0.5152, -0.9596, -0.6945, -1.0182, -0.7517,
         -0.6676, -0.5669,  3.5793, -0.2739, -0.7124, -0.5592,  4.0138, -0.8386,
         -0.6752, -0.9859, -0.4137, -1.0376]], device='cuda:0',
       grad_fn=<AddmmBackward>)
tensor([17], device='cuda:0')
tensor([[17]], device='cuda:0')
correct till now: 19
conv3d torch.Size([1, 128, 5, 179, 199])
reshape conv3d  torch.Size([1, 640, 179, 199])
torch.Size([1, 60])
tensor([[-1.1759, -0.8652, -0.2921, -0.8819, -0.3295, -1.4782, -1.0364, -0.3943,
         -0.6996, -0.4245, -0.3936, -0.3732, 

100%|██████████| 10/10 [02:47<00:00, 16.72s/it]

torch.Size([1, 60])
tensor([[-1.1667, -0.8221, -0.3064, -0.8526, -0.3225, -1.4581, -1.0547, -0.4111,
         -0.7216, -0.4593, -0.4207, -0.4425, -0.7720, -1.4451, -0.5979, -0.8860,
         -0.5508,  4.9576, -0.6389, -0.8320, -0.1962, -0.5996, -0.9202, -0.4076,
         -0.6193, -0.3924, -0.8987,  3.2379, -1.1977, -0.3483, -0.8610, -0.6937,
         -0.6825,  4.8160, -0.6428, -0.8730, -0.4212,  3.7697, -0.5033, -0.1236,
         -1.2888,  3.5428, -0.2018, -0.5483, -0.9748, -0.7131, -1.0549, -0.7972,
         -0.7045, -0.5258,  4.1161, -0.3039, -0.7300, -0.5467,  4.5207, -0.8707,
         -0.7402, -0.9832, -0.4475, -1.0337]], device='cuda:0',
       grad_fn=<AddmmBackward>)
tensor([17], device='cuda:0')
tensor([[17]], device='cuda:0')
correct till now: 22
Epoch: 9 , Training Accuracy: 22/11 (200%) Training Loss: 29.167876
